## Exercise Yamamoto Method

In this exercise the Yamamoto method will be applied and used as model parameter for the linear region of the steric mass action isotherm.

Import the data given in `./experiments/three_proteins` using the following helper function.
Then, visualize the data.

Define the parameters given in the table below as variables.

Consider the following parameters:
- length: $0.1~m$
- diameter: $7.7~mm$
- bed porosity: $0.36$
- particle radius: $34 \cdot 10^{-6}~m$
- particle porosity: $0.85$

Apply the Yamamoto method and save the return of the method in a variable. Print the characteristic charge and the equilibrium constant for all proteins in the system.

Visualize the results of the parameter estimation and check if the results are a good estimate of the experiments.

It can be recognized that at least one experiment is not in a good alignment with the found parameter.
Check the output of the parameter estimation and decide which experiment to neglect in the estimation.

Viewing the data, the third entry related to the third experiment does not fit into the ascending sequence for the peak salt concentration. Remove the third experiment from the parameter estimation and redo the estimation. Also, visualize the results.

# Bonus Task

Try to build a simple model which reproduces the experimental results.
The model should consist of an `Inlet`, the column model and an `Outlet`.
Use the values specified in the tables above.

Keep in mind the following steps for creating a model:
1. Specify missing UnitOperations
1. Construct the flow sheet
1. Construct the process
1. Simulation and plotting
1. Comparison to experiment

### 1. Specify missing UnitOperations

In [ ]:
from CADETProcess.processModel import Inlet, Outlet

inlet = Inlet(component_system, name='inlet')
inlet.flow_rate = 8.33e-09

outlet = Outlet(component_system, name='outlet')

### 2. Construct the flow sheet

In [ ]:
from CADETProcess.processModel import FlowSheet

flow_sheet = FlowSheet(component_system)
flow_sheet.add_unit(inlet)
flow_sheet.add_unit(column)
flow_sheet.add_unit(outlet)

flow_sheet.add_connection(inlet, column)
flow_sheet.add_connection(column, outlet)

### 3. Construct the process

In [ ]:
from CADETProcess.processModel import Process

flow_rate = 8.33e-09
sample_volume = 50e-9
wash_volume = 15e-6
gradient_volume =18.8e-6
strip_volume = 10e-6

wash_start = sample_volume/flow_rate
gradient_start = sample_volume/flow_rate + wash_volume/flow_rate
strip_start = sample_volume/flow_rate + wash_volume/flow_rate + gradient_volume/flow_rate
t_cycle = sample_volume/flow_rate + wash_volume/flow_rate + gradient_volume/flow_rate + strip_volume/flow_rate

slope = ((experiment_1.c_salt_end-experiment_1.c_salt_start)/(gradient_volume/flow_rate))

process = Process(flow_sheet, 'LWE_Lysozyme_18_8mL')
process.cycle_time = t_cycle

process.add_event('load', 'flow_sheet.inlet.c', [experiment_1.c_salt_start, 0.2, 0.24, 0.16], 0)
process.add_event('wash', 'flow_sheet.inlet.c', [experiment_1.c_salt_start, 0, 0, 0], wash_start)
process.add_event(
    'grad_start',
    'flow_sheet.inlet.c',
    [[experiment_1.c_salt_start, slope], [0, 0], [0, 0], [0, 0]],
    gradient_start
)
process.add_event('strip','flow_sheet.inlet.c', [experiment_1.c_salt_end, 0, 0, 0], strip_start)

### 7. Simulation and plotting

In [ ]:
from CADETProcess.simulator import Cadet

simulator = Cadet()
sim_res = simulator.simulate (process)

from CADETProcess.plotting import SecondaryAxis
sec = SecondaryAxis()
sec.components = ["Salt"]
sec.y_label = '$c_{Salt}$'
_ = sim_res.solution.column.outlet.plot(secondary_axis=sec)

### 8. Comparison to experiment

In [ ]:
from CADETProcess.comparison import Comparator, ReferenceIO

comparator = Comparator()
for i_p in range(experiment_1.n_proteins):
    Ref_name = 'Protein_'+str(i_p)
    comparator.add_reference(ReferenceIO(
        Ref_name, experiment_1.time, experiment_1.c_protein[:,i_p]
        )
    )
    comparator.add_difference_metric(
        'SSE', ReferenceIO(Ref_name, experiment_1.time, experiment_1.c_protein[:, i_p]),
        'outlet.outlet', components=component_system.labels[i_p+1]
    )

comparator.plot_comparison(sim_res, plot_individual=True)